In [1]:
%jsroot on

In [2]:
const double PPAC3_Z = -84.2-188;
const double PPAC2_Z = -206.2-188;
const double PPAC1_Z = -504.2-188;

In [3]:
gROOT->Macro("/home/zhuhy/JNBook/Experiment2022/taf/taf_E_theta/cuttemp_2H/taf_ad2_ac2A_d.c");
TCutG *taf_ad0_ac0A_d = (TCutG *)gROOT->GetListOfSpecials()->FindObject("taf_ad0_ac0A_d");
TCutG *C14_1 = (TCutG *)gROOT->GetListOfSpecials()->FindObject("C14_1");
TCutG *C14_2 = (TCutG *)gROOT->GetListOfSpecials()->FindObject("C14_2");
TCutG *C14_3 = (TCutG *)gROOT->GetListOfSpecials()->FindObject("C14_3");

In [4]:
int start = 450;
int stop = 450;
TChain *fch;
fch = new TChain("tree", "tele");
for (int i= start; i<=stop; i++){
  TString fileName = TString::Format("/data/d1/zhuhy/lanzhou2022/getdatappac/tele/tele%04d.root" , i);
  //TString fileName = TString::Format("/data/d1/zhuhy/telefile/tele%04d.root" , i);
  fch->AddFile(fileName.Data());}

TCanvas *c1=new TCanvas;

In [5]:
void SetTrace(TH2I *h,Double_t k,Double_t b){
        if(h==0) return;

        Int_t nbin  = h->GetNbinsX();
        Double_t zmin = h->GetXaxis()->GetXmin();
        Double_t zmax = h->GetXaxis()->GetXmax();
        for(int i=0;i<nbin;i++){
                Double_t nowz = zmin+(zmax-zmin)*i/nbin;
                Double_t nowf = nowz*k+b;
                h->Fill(nowz,nowf);
        }
}


In [6]:
   Double_t xzTrack[4] = {PPAC1_Z,PPAC2_Z,PPAC3_Z,100};
   Double_t yzTrack[4] = {PPAC1_Z,PPAC2_Z,PPAC3_Z,100};
   Double_t xTrack[4]={0,0,0,0};
   Double_t yTrack[4]={0,0,0,0};
   Double_t tx=0.,ty=0.;
   Double_t tp1x=0.,tp1y=0.;
   Double_t tp2x=0.,tp2y=0.;
   Double_t tp3x=0.,tp3y=0.;
   Double_t xchindf=0;
   Int_t countxc=0;
   Double_t ychindf=0;
   Int_t countyc=0;

   UShort_t t0_d1_xs[5],t0_d1_ys[5];
   UShort_t        t0d1_xhit;
   UShort_t        t0d1_yhit;
   int maxBin=0;
   Double_t t0_d1_xce[5],t0_d2_xce[5];
   Double_t xfpx[3],xfpy[3],xpCh[3][5][2];
   TBranch        *b_xfpx;   //!
   TBranch        *b_xfpy;   //!
   TBranch        *b_xpCh;   //!
   TBranch        *b_t0_d1_xs;   //!
   TBranch        *b_t0_d1_ys;   //!
   TBranch        *b_t0d1_xhit;   //!
   TBranch        *b_t0d1_yhit;   //!
   TBranch        *b_t0_d1_xce;   //!
   TBranch        *b_t0_d2_xce;   //!

        TH2I* hxz = new TH2I("hxz","Projection of X-Z",200,-750,200,200,-60,60);
        hxz->SetStats(kFALSE);
        TH2I* hyz = new TH2I("hyz","Projection of Y-Z",200,-750,200,200,-60,60);
        hyz->SetStats(kFALSE);

        TH2I* htarget = new TH2I("htarget","target",120,-30,30,120,-30,30);
        htarget->SetStats(kFALSE);

        TH2I* hit = new TH2I("hit","hit",64,0,64,64,0,64);
        hit->SetStats(kFALSE);

        TH1F *htargetx = new TH1F("htargetx","htargetx",120,-30,30);
        TH1F *htargety = new TH1F("htargety","htargety",120,-30,30);

        TH1F *p1xdt = new TH1F("p1xdt","p1xdt",400,-20,20);
        TH1F *p2xdt = new TH1F("p2xdt","p2xdt",400,-20,20);
        TH1F *p3xdt = new TH1F("p3xdt","p3xdt",400,-20,20);
        TH1F *p1ydt = new TH1F("p1ydt","p1ydt",400,-20,20);
        TH1F *p2ydt = new TH1F("p2ydt","p2ydt",400,-20,20);
        TH1F *p3ydt = new TH1F("p3ydt","p3ydt",400,-20,20);

        TH1F *xchi2 = new TH1F("xchi2","xchi2",100,0,10);
        TH1F *ychi2 = new TH1F("ychi2","ychi2",100,0,10);

        TGraph *gx[10];
        TGraph *gy[10];
        for(Int_t ii=0;ii<10;ii++){
            gx[ii]=new TGraph();
            gy[ii]=new TGraph();
        }

In [7]:
        Double_t ppacxoffset[3][809];
        Double_t ppacyoffset[3][809];

        for(Int_t ii=0;ii<3;ii++){
                for(Int_t jj=0;jj<809;jj++){
                        ppacxoffset[ii][jj]=0;
                        ppacyoffset[ii][jj]=0;
                }
        }


        ifstream read1;
        read1.open("/data/d1/zhuhy/lanzhou2022/getdatappac/paras/ppacoffset.txt", ios::in);

        if(!read1.is_open()){
                cout<<" Cannot find calikbParas"<<endl;
                return;
        }
        Int_t prunnum=0;
        while(!read1.eof()){
                read1>>prunnum;
                read1>>ppacxoffset[0][prunnum]>>ppacyoffset[0][prunnum]>>ppacxoffset[1][prunnum]>>ppacyoffset[1][prunnum]>>ppacxoffset[2][prunnum]>>ppacyoffset[2][prunnum];
        }
        read1.close();


In [8]:
   fch->SetBranchAddress("xfpx[3]", xfpx, &b_xfpx);
   fch->SetBranchAddress("xfpy[3]", xfpy, &b_xfpy);
   fch->SetBranchAddress("xpCh[3][5][2]", xpCh, &b_xpCh);
   fch->SetBranchAddress("t0_d1_xs[5]", t0_d1_xs, &b_t0_d1_xs);
   fch->SetBranchAddress("t0_d1_ys[5]", t0_d1_ys, &b_t0_d1_ys);
   fch->SetBranchAddress("t0d1_xhit", &t0d1_xhit, &b_t0d1_xhit);
   fch->SetBranchAddress("t0d1_yhit", &t0d1_yhit, &b_t0d1_yhit);
   fch->SetBranchAddress("t0_d1_xce[5]", t0_d1_xce, &b_t0_d1_xce);
   fch->SetBranchAddress("t0_d2_xce[5]", t0_d2_xce, &b_t0_d2_xce);

In [9]:
   Long64_t nentries = fch->GetEntries();
   TString fitname;
   for (Long64_t jentry=0; jentry<nentries;jentry++) {
        if(jentry%(200000)==0) cout<<jentry<<"/"<<nentries<<'\n';
        if(jentry>200000*20)break;
            fch->GetEntry(jentry);
       
            tx=0.;ty=0.;
            tp1x=0.;tp1y=0.;
            tp2x=0.;tp2y=0.;
            tp3x=0.;tp3y=0.;
            for(Int_t ii=0;ii<3;ii++){xTrack[ii]=0;yTrack[ii]=0;}
       
            for(Int_t ii=0;ii<3;ii++){
                xTrack[ii]=xpCh[ii][0][0]-xpCh[ii][1][0]-ppacxoffset[ii][start]-7;
                yTrack[ii]=xpCh[ii][3][0]-xpCh[ii][2][0]-ppacyoffset[ii][start]+3;   
            }
                xTrack[3]=(Double_t)((t0_d1_xs[0]*1.-32.));
                yTrack[3]=(Double_t)(t0_d1_ys[0]*1.-32.);
                if(t0_d2_xce[0]>216&&t0_d2_xce[0]<228&&t0_d1_xce[0]>159&&t0_d1_xce[0]>166&&abs(xpCh[0][0][0]-xpCh[0][1][0])<50&&abs(xpCh[1][0][0]-xpCh[1][1][0])<50&&abs(xpCh[2][0][0]-xpCh[2][1][0])<50&&t0d1_xhit==1&&t0d1_yhit==1&&t0_d1_xs[0]!=65535&&t0_d1_ys[0]!=65535){
                        TGraph *grYTrace = new TGraph(4,yzTrack,yTrack);
                        TGraph *grXTrace = new TGraph(3,xzTrack,xTrack);
                        fitname = TString::Format("[0]*(x-100.) + %f" , xTrack[3]);
                        TF1 *fXTrace = new TF1("fXTrace", fitname.Data(), -800, 200);            
                        TFitResultPtr xfitResult = grXTrace->Fit(fXTrace, "Sq");
                        if(xfitResult->Chi2()/(Double_t)(xfitResult->Ndf())<10){
                            if(countxc<7){
                                for(Int_t ii=0;ii<3;ii++)gx[countxc]->SetPoint(ii,xzTrack[ii],xTrack[ii]);
                                gx[countxc]->SetPoint(3,xzTrack[3],xTrack[3]);
                            }
                            xchindf=xchindf + xfitResult->Chi2()/(Double_t)(xfitResult->Ndf());
                            if(xfitResult->Chi2()!=0)xchi2->Fill(xfitResult->Chi2()/(Double_t)(xfitResult->Ndf()));
                            countxc++;
                            //cout<<"X: "<<xfitResult->Chi2()/(Double_t)(xfitResult->Ndf())<<'\n';
                        }
                        tx = fXTrace->Eval(0);
                        tp1x=-fXTrace->Eval(PPAC1_Z)+xTrack[0];p1xdt->Fill(tp1x);
                        tp2x=-fXTrace->Eval(PPAC2_Z)+xTrack[1];p2xdt->Fill(tp2x);
                        tp3x=-fXTrace->Eval(PPAC3_Z)+xTrack[2];p3xdt->Fill(tp3x);
                        SetTrace(hxz,fXTrace->GetParameter(0),fXTrace->GetParameter(0)*(xTrack[3]-100.));
                        delete grXTrace;
                        delete fXTrace;
                }
                if(t0_d2_xce[0]>216&&t0_d2_xce[0]<228&&t0_d1_xce[0]>159&&t0_d1_xce[0]>166&&abs(xpCh[0][3][0]-xpCh[0][2][0])<50&&abs(xpCh[1][3][0]-xpCh[1][2][0])<50&&abs(xpCh[2][3][0]-xpCh[2][2][0])<50&&t0d1_xhit==1&&t0d1_yhit==1&&t0_d1_xs[0]!=65535&&t0_d1_ys[0]!=65535){
                        TGraph *grYTrace = new TGraph(4,yzTrack,yTrack);
                        fitname = TString::Format("[0]*(x-100) + %f" , yTrack[3]);
                        TF1 *fYTrace = new TF1("fYTrace", fitname.Data(), -800, 200);
                        TFitResultPtr yfitResult = grYTrace->Fit(fYTrace, "Sq");
                        if(yfitResult->Chi2()/(Double_t)(yfitResult->Ndf())<10){
                            if(countyc<7){
                                for(Int_t ii=0;ii<3;ii++)gy[countyc]->SetPoint(ii,yzTrack[ii],yTrack[ii]);
                                gy[countyc]->SetPoint(3,yzTrack[3],yTrack[3]);
                            }
                            ychindf=ychindf + yfitResult->Chi2()/(Double_t)(yfitResult->Ndf());
                            if(yfitResult->Chi2()!=0)ychi2->Fill(yfitResult->Chi2()/(Double_t)(yfitResult->Ndf()));
                            countyc++;
                            //cout<<"Y: "<<yfitResult->Chi2()/(Double_t)(yfitResult->Ndf())<<'\n';
                        }
                        ty = fYTrace->Eval(0);
                        tp1y=-fYTrace->Eval(PPAC1_Z)+yTrack[0];p1ydt->Fill(tp1y);
                        tp2y=-fYTrace->Eval(PPAC2_Z)+yTrack[1];p2ydt->Fill(tp2y);
                        tp3y=-fYTrace->Eval(PPAC3_Z)+yTrack[2];p3ydt->Fill(tp3y);
                        SetTrace(hyz,fYTrace->GetParameter(0),fYTrace->GetParameter(0)*(yTrack[3]-100.));
                        delete grYTrace;
                        delete fYTrace;
                }
                if(tx!=0.&&ty!=0){
                    htarget->Fill(tx,ty);
                    htargetx->Fill(tx);
                    htargety->Fill(ty);
                }
   }

0/40727956
200000/40727956
400000/40727956
600000/40727956
800000/40727956
1000000/40727956
1200000/40727956
1400000/40727956
1600000/40727956
1800000/40727956
2000000/40727956
2200000/40727956
2400000/40727956
2600000/40727956
2800000/40727956
3000000/40727956
3200000/40727956
3400000/40727956
3600000/40727956
3800000/40727956
4000000/40727956


In [10]:
c1->Clear();
TH1F *frame = new TH1F("frame", ";X;Y", 1000, -800, 200);  // 设置 x 轴范围为 0 到 10
frame->SetMinimum(-20);  // 设置 y 轴最小值
frame->SetMaximum(20);  // 设置 y 轴最大值
frame->Draw();
for(Int_t ii=0;ii<7;ii++){
    gx[ii]->SetMarkerStyle(20);
    gx[ii]->SetMarkerSize(2);
    gx[ii]->SetMarkerColor(ii+1);
    gx[ii]->Fit("pol1","q");
    gx[ii]->Draw("samep");
}
c1->Draw();

In [11]:
c1->Clear();
TH1F *framey = new TH1F("framey", ";X;Y", 1000, -800, 200);  // 设置 x 轴范围为 0 到 10
framey->SetMinimum(-20);  // 设置 y 轴最小值
framey->SetMaximum(20);  // 设置 y 轴最大值
framey->Draw();
for(Int_t ii=0;ii<7;ii++){
    gy[ii]->SetMarkerStyle(20);
    gy[ii]->SetMarkerSize(2);
    gy[ii]->SetMarkerColor(ii+1);
    gy[ii]->Fit("pol1","q");
    gy[ii]->Draw("samep");
}
c1->Draw();

In [12]:
c1->Clear();
TF1 *gxchi2= new TF1("gxchi2","gaus");
maxbin = xchi2->GetMaximumBin();
xchi2->Fit("gxchi2","Q","",0+0.01*(maxbin-20),0+0.01*(maxbin+20));
xchi2->Draw();
c1->Draw();
cout<<"mean="<<gxchi2->GetParameter(1)<<'\n';

mean=0.0824573


In [13]:
c1->Clear();
TF1 *gychi2= new TF1("gychi2","gaus");
maxbin = ychi2->GetMaximumBin();
ychi2->Fit("gychi2","Q","",0+0.01*(maxbin-20),0+0.01*(maxbin+20));
ychi2->Draw();
c1->Draw();
cout<<"mean="<<gychi2->GetParameter(1)<<'\n';

mean=0.0792168


In [14]:
  //PPAC1 draw
  TLine *nppac[3];
  for(Int_t i=0;i<3;i++){
  nppac[i] = new TLine(xzTrack[i],-30,xzTrack[i],30);
  nppac[i]->SetLineWidth(5);
  nppac[i]->SetLineColor(1);
  }
  //Target draw
  TLine *target=new TLine(0,-15,0,15);
  target->SetLineWidth(5);
  target->SetLineColor(2);
  //Target draw
  TLine *t0=new TLine(111.76,-32,111.76,32);
  t0->SetLineWidth(5);
  t0->SetLineColor(1);

//100 111.76 123.52

In [15]:
Double_t mean[2][3];

In [16]:
c1->Clear();
TF1 *gp1xdt = new TF1("gp1xdt","gaus");
maxbin = p1xdt->GetMaximumBin();
p1xdt->Fit("gp1xdt","Q","",-20+0.1*(maxbin-5),-20+0.1*(maxbin+5));    
mean[0][0]=gp1xdt->GetParameter(1);
p1xdt->Draw();
c1->Draw();
cout<<"mean="<<gp1xdt->GetParameter(1)<<'\t'<<"sigma="<<gp1xdt->GetParameter(2)<<'\n';

mean=-0.0636708	sigma=0.383398


In [17]:
c1->Clear();
TF1 *gp2xdt = new TF1("gp2xdt","gaus");
maxbin = p2xdt->GetMaximumBin();
p2xdt->Fit("gp2xdt","Q","",-20+0.1*(maxbin-5),-20+0.1*(maxbin+5));   
mean[0][1]=gp2xdt->GetParameter(1);
p2xdt->Draw();
c1->Draw();
cout<<"mean="<<gp2xdt->GetParameter(1)<<'\t'<<"sigma="<<gp2xdt->GetParameter(2)<<'\n';

mean=0.20516	sigma=0.454145


In [18]:
c1->Clear();
TF1 *gp3xdt = new TF1("gp3xdt","gaus");
maxbin = p3xdt->GetMaximumBin();
p3xdt->Fit("gp3xdt","Q","",-20+0.1*(maxbin-5),-20+0.1*(maxbin+5));   
mean[0][2]=gp3xdt->GetParameter(1);
p3xdt->Draw();
c1->Draw();
cout<<"mean="<<gp3xdt->GetParameter(1)<<'\t'<<"sigma="<<gp3xdt->GetParameter(2)<<'\n';

mean=-0.03346	sigma=0.368394


In [19]:
c1->Clear();
TF1 *gp1ydt = new TF1("gp1ydt","gaus");
maxbin = p1ydt->GetMaximumBin();
p1ydt->Fit("gp1ydt","Q","",-20+0.1*(maxbin-5),-20+0.1*(maxbin+5));   
mean[1][0]=gp1ydt->GetParameter(1);
p1ydt->Draw();
c1->Draw();
cout<<"mean="<<gp1ydt->GetParameter(1)<<'\t'<<"sigma="<<gp1ydt->GetParameter(2)<<'\n';

mean=-0.0905534	sigma=0.313398


In [20]:
c1->Clear();
TF1 *gp2ydt = new TF1("gp2ydt","gaus");
maxbin = p2ydt->GetMaximumBin();
p2ydt->Fit("gp2ydt","Q","",-20+0.1*(maxbin-5),-20+0.1*(maxbin+5));   
mean[1][1]=gp2ydt->GetParameter(1);
p2ydt->Draw();
c1->Draw();
cout<<"mean="<<gp2ydt->GetParameter(1)<<'\t'<<"sigma="<<gp2ydt->GetParameter(2)<<'\n';

mean=0.160543	sigma=0.37774


In [21]:
c1->Clear();
TF1 *gp3ydt = new TF1("gp3ydt","gaus");
maxbin = p3ydt->GetMaximumBin();
p3ydt->Fit("gp3ydt","Q","",-20+0.1*(maxbin-5),-20+0.1*(maxbin+5));   
mean[1][2]=gp3ydt->GetParameter(1);
p3ydt->Draw();
c1->Draw();
cout<<"mean="<<gp3ydt->GetParameter(1)<<'\t'<<"sigma="<<gp3ydt->GetParameter(2)<<'\n';

mean=-0.0227701	sigma=0.315305


In [22]:
for(Int_t ii=0;ii<3;ii++){
    for(Int_t jj=0;jj<2;jj++){
        cout<<mean[jj][ii]<<" ";
    }
}

-0.0636708 -0.0905534 0.20516 0.160543 -0.03346 -0.0227701 

In [23]:
c1->Clear();
c1->SetLogz();
htarget->Draw("colz");
c1->Draw();

In [24]:
c1->Clear();
htargetx->Draw();
c1->Draw();

In [25]:
c1->Clear();
htargety->Draw();
c1->Draw();

In [26]:
c1->Clear();
c1->SetLogz();
hxz->Draw("colz");
for(Int_t i=0;i<3;i++)nppac[i]->Draw();
target->Draw();
t0->Draw();
c1->Draw();

In [27]:
c1->Clear();
c1->SetLogz();
hyz->Draw("colz");
for(Int_t i=0;i<3;i++)nppac[i]->Draw();
target->Draw();
t0->Draw();
c1->Draw();

\\\\##### 